In [9]:
from __future__ import annotations
import json
from typing import Literal, TypedDict
from thefuzz import fuzz
from pandas import Series, DataFrame
from constructor import DataframeConstructor
from interfaces import InnerPdData, LangNameData, OriginalCiaLanguageData
import pandas as pd
from FuzzySearcher import AggregateFuzzySearcher

from FuzzySearcher import FuzzySearcher, FuzzySearcherData


from typing import Any, TypedDict

df_constructor = DataframeConstructor()
class FuzzyDataGetter:
    
    def __init__(self, df: DataFrame, column: str, other_df: DataFrame, other_column: str):
        self.prefix = "fuzzy_"
        self.df = df
        self.column = column
        self.other_df = other_df
        self.other_column = other_column
        self.agg_fuzzy = AggregateFuzzySearcher.create(["ratio","token_set_ratio"])
        

    def get_fuzzy_data(self, row: Series[str]):
        other_keys = self.other_df[self.other_column].tolist()
        self.agg_fuzzy
        
        result = self.agg_fuzzy.run_against_multiple(row[self.column], other_keys)
        row[self.get_other_column_label()] = result.top_result.right
        row[self.get_other_column_similarity_label()] = result.top_result.similarity
        # row[self.get_other_column_fuzzy_matching_data_label()] = result.model_dump()
        
        return row

    def get_fuzzy_data_multiple_row(self, rows:list[Series[str]]):
       
        
       
        
        return rows
        

    def get_other_column_label(self):
        return f"{self.prefix}top_{self.other_column}"

    def get_other_column_similarity_label(self):
        return f"{self.get_other_column_label()}_similarity"

    def get_other_column_fuzzy_matching_data_label(self):
        return f"{self.get_other_column_label()}_fuzzy_matching_data"

    def apply(self):
        self.df = self.df.apply(self.get_fuzzy_data, axis=1)
        return self.df


# def init_pd_settings():
#     pd.set_option('display.max_rows', 100)
#     pd.set_option('display.max_columns', 20)
#     pd.set_option('display.width', None)
#     pd.set_option('display.max_colwidth', None)
# init_pd_settings()

lang_name_df = df_constructor.create_rest_api_language_name_dataframe()

new_cia_language_df = df_constructor.create_new_cia_language_dataframe()
original_wiki_language_df = df_constructor.create_wiki_language_df()

# Data cleaning

In [10]:
default_args = {
    "to_replace": [r"\bthe\b", r"\bof\b", r"\bThe\b", "[,()]", r"\d+", r"\[.*\]"],
    "value": "",
    "regex": True,
}

# clean data


lang_name_df["common"] = lang_name_df["common"].replace(**default_args)
lang_name_df["official"] = lang_name_df["official"].replace(**default_args)

new_cia_language_df["country"] = new_cia_language_df["country"].replace(**default_args)
original_wiki_language_df["country_or_region"] = original_wiki_language_df[
    "country_or_region"
].replace(**default_args)
original_wiki_language_df["regional_language"] = original_wiki_language_df[
    "regional_language"
].replace(**default_args)
original_wiki_language_df["official_language"] = original_wiki_language_df[
    "official_language"
].replace(**default_args)
original_wiki_language_df["national_language"] = original_wiki_language_df[
    "national_language"
].replace(**default_args)

In [11]:
# tokenize and apply cleaning strategy
tokenized:Series[Any] = original_wiki_language_df["official_language"].str.split(" ")
def clean(str_arr: list[str]) -> list[str]:
    result:list[str] = []
    for str in str_arr:
        if len(str) == 0:
            continue
        if str[0].islower():
            continue
        if str.lower() == "none":
            continue
        result.append(str)
    return result
    

# commit changes
tokenized = tokenized.apply(clean)
original_wiki_language_df["primary_language"] = tokenized.apply(lambda x: x[0] if len(x) > 0 else None)


# Fuzzy matching score calculations

In [12]:
# cia to wiki

getter = FuzzyDataGetter(
    df=new_cia_language_df,
    column="country",
    other_column="country_or_region",
    other_df=original_wiki_language_df,
)
new_cia_language_df = getter.apply()


In [13]:
# rest countries to cia, taking into account both the common and official name, and using the maximum of the 2 for the final score

getter1 = FuzzyDataGetter(
    df=lang_name_df,
    column="official",
    other_column="country",
    other_df=new_cia_language_df,
)
getter1.prefix = "fuzzy_official_"
lang_name_df = getter1.apply()


getter2 = FuzzyDataGetter(
    df=lang_name_df,
    column="common",
    other_column="country",
    other_df=new_cia_language_df,
)
getter2.prefix = "fuzzy_common_"
lang_name_df = getter2.apply()
lang_name_df['max_official_score_between_common_and_official'] = lang_name_df[[getter1.get_other_column_similarity_label(), getter2.get_other_column_similarity_label()]].max(axis=1)

# Joining tables based on their highest calculated score
- Whether or not to use the country based on calculated score can be determined later in queries in the DB
- Rest countries will be used as the primary table for the joins
- The scoring for the fuzzy join between the rest countries table and the wiki table directly is missing, but can be added later in future processing

In [14]:
# rest countries data to cia data
lang_name_df = lang_name_df.merge(new_cia_language_df, left_on="fuzzy_common_top_country", right_on="country", how="left", suffixes=(None,"_cia"))
lang_name_df

,id,common,official,nativeName,fuzzy_official_top_country,fuzzy_official_top_country_similarity,fuzzy_common_top_country,fuzzy_common_top_country_similarity,max_official_score_between_common_and_official,country,primary_language,fuzzy_top_country_or_region,fuzzy_top_country_or_region_similarity
0,2,Afghanistan,Islamic Republic Afghanistan,"{'prs': {'common': 'افغانستان', 'official': 'ج...",Afghanistan,100,Afghanistan,100,100,Afghanistan,English,Afghanistan,100
1,3,Angola,Republic Angola,"{'por': {'common': 'Angola', 'official': 'Repú...",Angola,100,Angola,100,100,Angola,Portuguese,Angola,100
2,5,Åland Islands,Åland Islands,"{'swe': {'common': 'Åland', 'official': 'Lands...",Cayman Islands,77,Cayman Islands,77,77,Cayman Islands,English,Cook Islands,74
3,6,Albania,Republic Albania,"{'sqi': {'common': 'Shqipëria', 'official': 'R...",Albania,100,Albania,100,100,Albania,Greek,Albania,100
4,7,Andorra,Principality Andorra,"{'cat': {'common': 'Andorra', 'official': 'Pri...",Andorra,100,Andorra,100,100,Andorra,Portuguese,Andorra,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,248,South Africa,Republic South Africa,"{'afr': {'common': 'South Africa', 'official':...",South Africa,100,South Africa,100,100,South Africa,English,South Africa,100
246,249,Zambia,Republic Zambia,"{'eng': {'common': 'Zambia', 'official': 'Repu...",Zambia,100,Zambia,100,100,Zambia,English,Zambia,100
247,250,Zimbabwe,Republic Zimbabwe,"{'bwg': {'common': 'Zimbabwe', 'official': 'Re...",Zimbabwe,100,Zimbabwe,100,100,Zimbabwe,English,Zimbabwe,100
248,1,Aruba,Aruba,"{'nld': {'common': 'Aruba', 'official': 'Aruba...",Aruba,100,Aruba,100,100,Aruba,Portuguese,Cuba,67


In [15]:
# rest countries data to wiki data
lang_name_df = lang_name_df.merge(original_wiki_language_df, left_on="fuzzy_top_country_or_region", right_on="country_or_region", how="left", suffixes=(None,"_wiki"))
lang_name_df = lang_name_df

In [16]:
lang_name_df.to_json("clean_data.json", orient="records")